In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler 
import pandas as pd 
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import time
import sys
%matplotlib inline
import random
import copy
import logging
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import keras.backend as K
import tensorflow as tf
from tsfresh import extract_features
import tsfresh.feature_extraction.feature_calculators as features
from pandas import DataFrame
import FFNN_Function
import LSTM_Function

# Setup logging.
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='meta-learning.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

Serie_names = ['Serie 1','Serie 2','Serie 3','Serie 4','Serie 5','Serie 6','Serie 7','Serie 8','Serie 9','Serie 10','Serie 11','Serie 12']
tree_examples = DataFrame (columns  = ['FFNN','ARIMA','LSTM','length','kurtosis','energy','variance'], index = Serie_names)



Using TensorFlow backend.


In [2]:
Series = pd.read_excel("NN3_FINAL_DATASET_WITH_TEST_DATA.xlsx", sheet_name="Hoja 1")
Series

,Serie 1,Serie 2,Serie 3,Serie 4,Serie 5,Serie 6,Serie 7,Serie 8,Serie 9,Serie 10,Serie 11,Serie 12,Unnamed: 12
0,4998,5033.0,5304,685.0,5006.9,4950,4318,1740.0,6318.0,2605,3140,4207.354924,1
1,4480,4509.5,4264,656.0,5165.1,4900,4224,1005.0,5991.0,2510,3270,4546.487134,2
2,4824,3970.0,3224,1288.0,4975.5,5500,4014,1792.5,6405.0,3520,3690,705.600040,3
3,4814,3378.0,2400,3914.0,4983.6,5550,3620,1965.0,6230.0,3200,4360,-3784.012477,4
4,4602,2866.0,1968,6837.0,5105.4,5900,3512,2085.0,6111.0,2550,3520,-4794.621373,5
5,4499,2315.5,1532,7104.0,5025.5,5150,3590,2440.0,5317.0,2640,4140,-1397.077491,6
6,4594,1895.0,1000,7464.0,4847.8,5700,3700,2317.5,4893.0,2280,3440,3284.932994,7
7,4600,8401.5,55024,6308.0,4963.3,6450,3752,2312.5,4867.0,2620,3750,4946.791233,8
8,4507,8040.0,53804,5549.0,4796.0,4800,3866,2667.5,4785.0,1940,4140,2060.592426,9
9,4606,7534.0,42540,6365.0,5042.0,6300,3892,3655.0,5099.0,1415,5610,-2720.105554,10


In [3]:
steps_ahead = 1
test_size = 14
Serie = Series['Serie 1'].astype(float)
Serie = Serie.dropna()
Serie = Serie.values
Serie = Serie.reshape(Serie.shape[0],-1)
scaler = MinMaxScaler(feature_range = (0.1, 0.9))
Serie = scaler.fit_transform(Serie)  
FFNN_Function.main(steps_ahead,test_size,Serie,'Serie 1')[0].mse

  0%|          | 0/10 [00:00<?, ?it/s]

((118, 12), (118, 1)) ((2, 12), (2, 1))
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
[0.0068579548969864845, 0.0068579548969864845]


 10%|█         | 1/10 [00:03<00:27,  3.10s/it]

((126, 4), (126, 1)) ((10, 4), (10, 1))
[0.016128763556480408, 0.016128763556480408]


 20%|██        | 2/10 [00:06<00:25,  3.20s/it]

...

 90%|█████████ | 9/10 [00:29<00:03,  3.77s/it]

((124, 6), (124, 1)) ((8, 6), (8, 1))
[0.02719997987151146, 0.02719997987151146]


  0%|          | 0/10 [00:00<?, ?it/s]

((118, 12), (118, 1)) ((2, 12), (2, 1))


KeyboardInterrupt: 